# import

In [2]:
import pandas as pd 
import numpy as np

# file save
import json 

# regular expression
import re 

# selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException

In [4]:
# File path to the JSON file
file_path = "pokemonTCG_links.json"

# Open the JSON file and load its contents into a variable
with open(file_path, 'r') as file:
    links = json.load(file)

In [38]:

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up Chrome driver service
service = Service('/path/to/chromedriver')  # Replace with the actual path to chromedriver

# Start the Chrome webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

data = []  # Initialize the data list

for key, link in links.items():
    for i in range(len(link)):
        if i < len(link):  # Check if index is within the valid range
            # Load the webpage
            driver.get(link[i])

            # Wait for the page to load (you may need to adjust the waiting time depending on the page)
            driver.implicitly_wait(10)  # Wait for 10 seconds

            # Find the table element using the XPath
            table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[5]/tbody/tr/td[1]/table')

            # Get the HTML source of the table element
            table_html = table_element.get_attribute('innerHTML')

            # Create a BeautifulSoup object from the HTML source
            soup = BeautifulSoup(table_html, 'html.parser')

            # Find the serial numbers from a table/html
            numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})
            numbers_filtered = [num.text.strip() for num in numbers if num.text.strip().endswith("/100")]

            # Find card names through HTML
            items = soup.find_all('td', attrs={'style': 'background:#FFFFFF'})
            cardNames = []
            for item in items:
                if item.find('a') and '/100' not in item.text:
                    card_name = item.find('a').text.strip()
                    cardNames.append(card_name)
            cardNames = list(filter(lambda x: x != '', cardNames))

            # Find types
            types = soup.find_all('th', attrs={'align': 'center'})
            pTypes = []
            for i in range(len(types)):
                html = types[i]
                if 70 <= i <= 86:
                    title = html.text.strip()
                    pTypes.append(title)
                else:
                    a_tags = html.find_all('a')
                    num_a_tags = len(a_tags)
                    if num_a_tags >= 2:
                        type1 = a_tags[0]
                        title1 = type1['title']
                        type2 = a_tags[1]
                        title2 = type2['title']
                        pTypes.append([title1, title2])
                    elif num_a_tags == 1:
                        a_tag = a_tags[0]
                        title = a_tag['title']
                        pTypes.append(title)
                    else:
                        print(f"Error: Insufficient 'a' tags for element {i}")

            # Finding rarity string
            image_anchors = soup.find_all('a', class_='image')
            possible_titles = ['Rare Holo', 'Rare', 'Uncommon', 'Common', 'Rare Holo ex', 'Shiny Rare', 'ShinyRare Holo']
            Rarity = []
            for anchor in image_anchors:
                title = anchor.get('title')
                if title in possible_titles:
                    Rarity.append(title)

            # Append the extracted data to the data list
            data.append({
                'serial_number': numbers_filtered,
                'name': cardNames,
                'type': pTypes,
                'rarity': Rarity,
                'expansion_name': key,
                'set_name': link[i] if i < len(link) else ''
            })
            
            # Save the data to a CSV file every 100 iterations
            if len(data) % 10 == 0:
                df = pd.DataFrame(data)  # Create a DataFrame from the data list
                df.to_csv('pokemon_data.csv', index=False)  # Save the DataFrame to a CSV file


# Close the Chrome webdriver
driver.quit()

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


Error: Insufficient 'a' tags for element 69
Error: Insufficient 'a' tags for element 87
Error: Insufficient 'a' tags for element 88
Error: Insufficient 'a' tags for element 89
Error: Insufficient 'a' tags for element 90
Error: Insufficient 'a' tags for element 91
Error: Insufficient 'a' tags for element 92
Error: Insufficient 'a' tags for element 93
Error: Insufficient 'a' tags for element 94
Error: Insufficient 'a' tags for element 63
Error: Insufficient 'a' tags for element 57
Error: Insufficient 'a' tags for element 58
Error: Insufficient 'a' tags for element 59
Error: Insufficient 'a' tags for element 60
Error: Insufficient 'a' tags for element 61
Error: Insufficient 'a' tags for element 14
Error: Insufficient 'a' tags for element 15
Error: Insufficient 'a' tags for element 16
Error: Insufficient 'a' tags for element 17
Error: Insufficient 'a' tags for element 18
Error: Insufficient 'a' tags for element 19
Error: Insufficient 'a' tags for element 99
Error: Insufficient 'a' tags for

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.453
  (Session info: headless chrome=113.0.5672.126)
Stacktrace:
0   chromedriver                        0x0000000104df0598 chromedriver + 4973976
1   chromedriver                        0x0000000104de7913 chromedriver + 4938003
2   chromedriver                        0x00000001049a4cd6 chromedriver + 470230
3   chromedriver                        0x000000010498c94c chromedriver + 371020
4   chromedriver                        0x000000010498c5c6 chromedriver + 370118
5   chromedriver                        0x000000010498ae4a chromedriver + 364106
6   chromedriver                        0x000000010498b39c chromedriver + 365468
7   chromedriver                        0x000000010499a3cb chromedriver + 426955
8   chromedriver                        0x000000010499bd22 chromedriver + 433442
9   chromedriver                        0x00000001049b0d84 chromedriver + 519556
10  chromedriver                        

In [40]:
data = pd.read_csv('pokemon_data.csv')

In [41]:
data

,serial_number,name,type,rarity,expansion_name,set_name
0,[],"['Alakazam', 'Blastoise', 'Chansey', 'Charizar...","['Psychic', 'Water', 'Colorless', 'Fire', 'Col...","['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare ...",Original_Series,NaN
1,[],"['Clefable', 'Electrode', 'Flareon', 'Jolteon'...","['Colorless', 'Lightning', 'Fire', 'Lightning'...","['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare ...",Original_Series,NaN
2,[],"['Aerodactyl', 'Articuno', 'Ditto', 'Dragonite...","['Fighting', 'Water', 'Colorless', 'Colorless'...","['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare ...",Original_Series,NaN
3,[],[],[],[],Original_Series,https://bulbapedia.bulbagarden.net/wiki/Base_S...
4,[],"['Dark Alakazam', 'Dark Arbok', 'Dark Blastois...","['Psychic', 'Grass', 'Water', 'Fire', 'Colorle...","['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare ...",Original_Series,NaN
5,[],[],[],[],Original_Series,https://bulbapedia.bulbagarden.net/wiki/Gym_He...
6,[],"[""Blaine's Arcanine"", ""Blaine's Charizard"", ""B...","['Fire', 'Fire', 'Fire', 'Grass', 'Water', 'Fi...","['Rare Holo', 'Rare Holo', 'Rare Holo', 'Rare ...",Original_Series,NaN
7,[],[],[],[],Neo_Series,https://bulbapedia.bulbagarden.net/wiki/Neo_Ge...
8,[],[],[],[],Neo_Series,https://bulbapedia.bulbagarden.net/wiki/Neo_Di...
9,[],[],[],[],Neo_Series,https://bulbapedia.bulbagarden.net/wiki/Neo_Re...


In [79]:
link[1]

["https://bulbapedia.bulbagarden.net/wiki/Champion's_Path_(TCG)#Set_lists",
 'https://bulbapedia.bulbagarden.net/wiki/Shining_Fates_(TCG)#Set_lists',
 'https://bulbapedia.bulbagarden.net/wiki/Celebrations_(TCG)#Set_lists',
 'https://bulbapedia.bulbagarden.net/wiki/Pokémon_GO_(TCG)#Set_lists',
 'https://bulbapedia.bulbagarden.net/wiki/Crown_Zenith_(TCG)#Set_lists']

In [ ]:
# Load the webpage
            driver.get(link[i])

            # Wait for the page to load (you may need to adjust the waiting time depending on the page)
            driver.implicitly_wait(10)  # Wait for 10 seconds

            # Find the table element using the XPath
            table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[5]/tbody/tr/td[1]/table')

            # Get the HTML source of the table element
            table_html = table_element.get_attribute('innerHTML')

            # Create a BeautifulSoup object from the HTML source
            soup = BeautifulSoup(table_html, 'html.parser')

            # Find the serial numbers from a table/html
            numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})
            numbers_filtered = [num.text.strip() for num in numbers if num.text.strip().endswith("/100")]

            # Find card names through HTML
            items = soup.find_all('td', attrs={'style': 'background:#FFFFFF'})
            cardNames = []
            for item in items:
                if item.find('a') and '/100' not in item.text:
                    card_name = item.find('a').text.strip()
                    cardNames.append(card_name)
            cardNames = list(filter(lambda x: x != '', cardNames))

            # Find types
            types = soup.find_all('th', attrs={'align': 'center'})
            pTypes = []
            for i in range(len(types)):
                html = types[i]
                if 70 <= i <= 86:
                    title = html.text.strip()
                    pTypes.append(title)
                else:
                    a_tags = html.find_all('a')
                    num_a_tags = len(a_tags)
                    if num_a_tags >= 2:
                        type1 = a_tags[0]
                        title1 = type1['title']
                        type2 = a_tags[1]
                        title2 = type2['title']
                        pTypes.append([title1, title2])
                    elif num_a_tags == 1:
                        a_tag = a_tags[0]
                        title = a_tag['title']
                        pTypes.append(title)
                    else:
                        print(f"Error: Insufficient 'a' tags for element {i}")

            # Finding rarity string
            image_anchors = soup.find_all('a', class_='image')
            possible_titles = ['Rare Holo', 'Rare', 'Uncommon', 'Common', 'Rare Holo ex', 'Shiny Rare', 'ShinyRare Holo']
            Rarity = []
            for anchor in image_anchors:
                title = anchor.get('title')
                if title in possible_titles:
                    Rarity.append(title)

            # Append the extracted data to the data list
            data.append({
                'serial_number': numbers_filtered,
                'name': cardNames,
                'type': pTypes,
                'rarity': Rarity,
                'expansion_name': key,
                'set_name': link[i] if i < len(link) else ''
            })
            
            # Save the data to a CSV file every 100 iterations
            if len(data) % 10 == 0:
                df = pd.DataFrame(data)  # Create a DataFrame from the data list
                df.to_csv('pokemon_data.csv', index=False)  # Save the DataFrame to a CSV file


# Close the Chrome webdriver
driver.quit()

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

In [84]:
for key, link in enumerate(links.items()):
    
    for i in range(0,len(link[1])):
        query = link[1][i]
        
        
        # Load the webpage
        driver.get(query)

        # Wait for the page to load (you may need to adjust the waiting time depending on the page)
        driver.implicitly_wait(10)  # Wait for 10 seconds

        # Find the table element using the XPath
        table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/table[5]/tbody/tr/td[1]/table')

        # Get the HTML source of the table element
        table_html = table_element.get_attribute('innerHTML')

        # Create a BeautifulSoup object from the HTML source
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # Find the serial numbers from a table/html
        numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})
        numbers_filtered = [num.text.strip() for num in numbers if num.text.strip().endswith("/100")]


        print(numbers_filtered)
        
        
        



MaxRetryError: HTTPConnectionPool(host='localhost', port=59100): Max retries exceeded with url: /session/1a86a8df5cb2fb69b65a193104c073b2/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdf5ba46ee0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [85]:
link[1][i]


'https://bulbapedia.bulbagarden.net/wiki/Base_Set_(TCG)#Set_lists'